# Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## GroupBy Mechanics

In [4]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.007189,0.886429
1,a,two,-1.296221,-2.001637
2,b,one,0.274992,-0.371843
3,b,two,0.228913,1.669025
4,a,one,1.352917,-0.438570


In [5]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [6]:
grouped.mean()

key1
a    0.354628
b    0.251952
Name: data1, dtype: float64

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     1.180053
      two    -1.296221
b     one     0.274992
      two     0.228913
Name: data1, dtype: float64

In [8]:
means.unstack()

key2,one,two
key1,,
a,1.180053,-1.296221
b,0.274992,0.228913


In [9]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -1.296221
            2006    0.274992
Ohio        2005    0.618051
            2006    1.352917
Name: data1, dtype: float64

In [10]:
df.groupby('key1').mean()
df.groupby(['key1', 'key2']).mean()

C:\Users\sagar\AppData\Local\Temp/ipykernel_3016/896555946.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('key1').mean()


data1     data2
key1 key2                    
a    one   1.180053  0.223930
     two  -1.296221 -2.001637
b    one   0.274992 -0.371843
     two   0.228913  1.669025

In [11]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [12]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.007189  0.886429
1    a  two -1.296221 -2.001637
4    a  one  1.352917 -0.438570
b
  key1 key2     data1     data2
2    b  one  0.274992 -0.371843
3    b  two  0.228913  1.669025


In [13]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.007189  0.886429
4    a  one  1.352917 -0.438570
('a', 'two')
  key1 key2     data1     data2
1    a  two -1.296221 -2.001637
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.274992 -0.371843
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.228913  1.669025


In [14]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.274992,-0.371843
3,b,two,0.228913,1.669025


In [15]:
df.dtypes
grouped = df.groupby(df.dtypes, axis=1)

In [16]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.007189  0.886429
1 -1.296221 -2.001637
2  0.274992 -0.371843
3  0.228913  1.669025
4  1.352917 -0.438570
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [17]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.223930
     two  -2.001637
b    one  -0.371843
     two   1.669025

In [18]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped
s_grouped.mean()

key1  key2
a     one     0.223930
      two    -2.001637
b     one    -0.371843
      two     1.669025
Name: data2, dtype: float64

### Grouping with Dicts and Series

In [20]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,0.286350,0.377984,-0.753887,0.331286,1.349742
Steve,0.069877,0.246674,-0.011862,1.004812,1.327195
Wes,-0.919262,NaN,NaN,0.758363,-0.660524
Jim,0.862580,-0.010032,0.050009,0.670216,0.852965
Travis,-0.955869,-0.023493,-2.304234,-0.652469,-1.218302


In [22]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [23]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-0.422601,2.014076
Steve,0.992950,1.643745
Wes,0.758363,-1.579786
Jim,0.720225,1.705513
Travis,-2.956703,-2.197664


In [24]:
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

In [25]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.229668,0.367952,-0.703877,1.759864,1.542183
5,0.069877,0.246674,-0.011862,1.004812,1.327195
6,-0.955869,-0.023493,-2.304234,-0.652469,-1.218302


In [26]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.919262  0.377984 -0.753887  0.331286 -0.660524
  two  0.862580 -0.010032  0.050009  0.670216  0.852965
5 one  0.069877  0.246674 -0.011862  1.004812  1.327195
6 two -0.955869 -0.023493 -2.304234 -0.652469 -1.218302

### Grouping by Index Levels

In [27]:

columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -1.332610  1.074623  0.723642  0.690002  1.001543
1     -0.503087 -0.622274 -0.921169 -0.726213  0.222896
2      0.051316 -1.157719  0.816707  0.433610  1.010737
3      1.824875 -0.997518  0.850591 -0.131578  0.912414

In [28]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

In [29]:
df
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.283771
b    0.270384
Name: data1, dtype: float64

In [30]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

C:\Users\sagar\AppData\Local\Temp/ipykernel_3016/663192242.py:3: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)


,data1,data2
key1,,
a,2.649138,2.888067
b,0.046079,2.040868


In [31]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.354628  1.440090 -1.296221 -0.144516  1.007189  1.180053   
b      2.0  0.251952  0.032583  0.228913  0.240433  0.251952  0.263472   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.352917   3.0 -0.517926  1.445668 -2.001637 -1.220104 -0.438570   
b     0.274992   2.0  0.648591  1.443111 -0.371843  0.138374  0.648591   

                          
           75%       max  
key1                      
a     0.223930  0.886429  
b     1.158808  1.669025

### Column-Wise and Multiple Function Application

In [34]:
tips = pd.read_csv('examples/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

FileNotFoundError: ignored

In [35]:
grouped = tips.groupby(['day', 'smoker'])

NameError: ignored

In [ ]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

In [ ]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

In [ ]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

In [ ]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

In [ ]:
result['tip_pct']

In [ ]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

In [ ]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

### Returning Aggregated Data Without Row Indexes

In [ ]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

## Apply: General split-apply-combine

In [ ]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

In [ ]:
tips.groupby('smoker').apply(top)

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()
result
result.unstack('smoker')

f = lambda x: x.describe()
grouped.apply(f)

### Suppressing the Group Keys

In [ ]:
tips.groupby('smoker', group_keys=False).apply(top)

### Quantile and Bucket Analysis

In [36]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-1.23, 0.489]
1    (0.489, 2.208]
2    (-1.23, 0.489]
3    (0.489, 2.208]
4    (-1.23, 0.489]
5    (0.489, 2.208]
6    (-1.23, 0.489]
7    (-1.23, 0.489]
8    (0.489, 2.208]
9    (-1.23, 0.489]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-2.956, -1.23] < (-1.23, 0.489] < (0.489, 2.208] < (2.208, 3.928]]

In [37]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.956, -1.23]",-3.399312,1.670835,96.0,-0.041782
"(-1.23, 0.489]",-2.989741,3.260383,601.0,-0.010093
"(0.489, 2.208]",-3.745356,2.954439,292.0,0.103729
"(2.208, 3.928]",-1.929776,1.765640,11.0,0.055158


In [38]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-3.399312,1.670835,100.0,-0.044313
1,-2.153545,2.628441,100.0,-0.019363
2,-2.925113,2.527939,100.0,-0.097075
3,-2.315555,3.260383,100.0,0.076381
4,-2.047939,2.074345,100.0,-0.105892
5,-2.989741,2.184810,100.0,0.087717
6,-2.223506,2.458842,100.0,-0.035383
7,-3.056990,2.954439,100.0,-0.047818
8,-3.745356,2.735527,100.0,0.158214


### Example: Filling Missing Values with Group-Specific       Values

In [39]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s
s.fillna(s.mean())

0   -0.246039
1    0.425384
2   -0.246039
3   -0.296336
4   -0.246039
5   -0.867165
dtype: float64

In [40]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.100440
New York      1.018334
Vermont      -1.266473
Florida       0.166839
Oregon        0.133483
Nevada        0.672024
California    0.546705
Idaho         0.779001
dtype: float64

In [41]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data
data.groupby(group_key).mean()

East    0.361578
West    0.340094
dtype: float64

In [44]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

C:\Users\sagar\AppData\Local\Temp/ipykernel_3016/649295739.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_mean)


Ohio         -0.100440
New York      1.018334
Vermont       0.361578
Florida       0.166839
Oregon        0.133483
Nevada        0.340094
California    0.546705
Idaho         0.340094
dtype: float64

In [45]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

C:\Users\sagar\AppData\Local\Temp/ipykernel_3016/2621590873.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_func)


Ohio         -0.100440
New York      1.018334
Vermont       0.500000
Florida       0.166839
Oregon        0.133483
Nevada       -1.000000
California    0.546705
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [46]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [47]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [48]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

3C      3
3S      3
10D    10
9S      9
10H    10
dtype: int64

In [49]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  AC      1
   3C      3
D  5D      5
   4D      4
H  10H    10
   7H      7
S  QS     10
   7S      7
dtype: int64

In [50]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

10C    10
8C      8
QD     10
2D      2
3H      3
4H      4
7S      7
8S      8
dtype: int64

### Example: Group Weighted Average and Correlation

In [51]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.100120,0.623944
1,a,0.567399,0.050591
2,a,0.565656,0.622580
3,a,-0.396560,0.431732
4,b,-0.146417,0.433396
5,b,0.669297,0.074403
6,b,1.238337,0.577881
7,b,0.782353,0.290255


In [52]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.157407
b    0.675202
dtype: float64

In [53]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.info()
close_px[-4:]

FileNotFoundError: ignored

In [54]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [55]:
rets = close_px.pct_change().dropna()

NameError: ignored

In [56]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

NameError: ignored

In [57]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

NameError: ignored

### Example: Group-Wise Linear Regression

In [58]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

ModuleNotFoundError: ignored

In [59]:
by_year.apply(regress, 'AAPL', ['SPX'])

NameError: ignored

## Pivot Tables and Cross-Tabulation

In [60]:
tips.pivot_table(index=['day', 'smoker'])

NameError: ignored

In [61]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

NameError: ignored

In [62]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

NameError: ignored

In [63]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

NameError: ignored

In [64]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

NameError: ignored

### Cross-Tabulations: Crosstab

In [68]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [69]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [70]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [71]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

NameError: ignored

In [72]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion